# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn import set_config
set_config(transform_output='pandas')

tf.keras.utils.set_random_seed(42)
tf.random.set_seed(42)
np.random.seed(42)

tf.config.experimental.enable_op_determinism()

# Data
- Making a mulit-class text classification to match each passage to the correct author.
- ['Spooky'](https://drive.google.com/file/d/1uUtuiAM1AqGvDMrd3-L8vLFfHPrt3pWG/view?usp=sharing) dataset.
    - Passages from three famaous authors:
        - Edgar Allan Poe
        - HP Lovecraft
        - Mary Shelley

In [2]:
# Loading data from data folder
df = pd.read_csv('Data/spooky.csv', index_col='id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19579 entries, id26305 to id00393
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    19579 non-null  object
 1   author  19579 non-null  object
dtypes: object(2)
memory usage: 458.9+ KB


- Want to look at the length of each text
- Using lambda function to split on space and get the length
- Using .describe to get statics on length

In [3]:
# Making new column to hold text length
df['sequence_length'] = df['text'].map(lambda x: len(x.split(' ')))
df.head()

,text,author,sequence_length
id,,,
id26305,"This process, however, afforded me no means of...",EAP,41
id17569,It never once occurred to me that the fumbling...,HPL,14
id11008,"In his left hand was a gold snuff box, from wh...",EAP,36
id27763,How lovely is spring As we looked from Windsor...,MWS,34
id12958,"Finding nothing else, not even gold, the Super...",HPL,27


In [4]:
df['sequence_length'].describe()

count    19579.000000
mean        26.730477
std         19.048353
min          2.000000
25%         15.000000
50%         23.000000
75%         34.000000
max        861.000000
Name: sequence_length, dtype: float64

- Mean length = 26.7
- Range length = 2-861

# RandomUnderSampler

In [5]:
df['author'].value_counts(dropna=False)

EAP    7900
MWS    6044
HPL    5635
Name: author, dtype: int64

In [6]:
# Resampling dataframe and not X,y
sampler = RandomUnderSampler()
df,_=sampler.fit_resample(df,df['author'])
df['author'].value_counts(dropna=False)

EAP    5635
HPL    5635
MWS    5635
Name: author, dtype: int64

# X, y
- Using .map to change 'author' column to numerical data and defineing it as y(target)

In [7]:
# Creat a map for targets
target_map = {'EAP':0, 'HPL':1, 'MWS':2}

# Define y as target
y = df['author'].map(target_map)
y

id
id22483    0
id18809    0
id16322    0
id13423    0
id09553    0
          ..
id22356    2
id11504    2
id00149    2
id16796    2
id23776    2
Name: author, Length: 16905, dtype: int64

In [8]:
# X
X = df['text'].values
X

array(['He then asked me, suddenly, if I had observed any thing peculiar at the scene of the atrocity.',
       'Through the exertions of Beauvais, the matter was industriously hushed up, as far as possible; and several days had elapsed before any public emotion resulted.',
       'The cold was intense, and obliged me to wrap up closely in an overcoat.',
       ...,
       'What I ask of you is reasonable and moderate; I demand a creature of another sex, but as hideous as myself; the gratification is small, but it is all that I can receive, and it shall content me.',
       'The birth of her daughter, embryo copy of her Raymond, filled up the measure of her content, and produced a sacred and indissoluble tie between them.',
       "I heard my sister's sobs, and thought, happy are women who can weep, and in a passionate caress disburthen the oppression of their feelings; shame and habitual restraint hold back a man."],
      dtype=object)

In [9]:
classes = y.unique()
classes

array([0, 1, 2])

# Convert to dataset object using Dataset.from_tensor_slices()
- Shuffle the data

In [10]:
# convert to Dataset object
ds = tf.data.Dataset.from_tensor_slices((X,y))

# Shuffle dataset
ds = ds.shuffle(buffer_size = len(ds), reshuffle_each_iteration=False)

Metal device set to: Apple M1


# TTS
- train size = .7
- val size = .2
- test size = 1 - (train + val)
- Use batch size 31

In [11]:
# Set the ratio of train, val, and test split
split_train = .7
split_val = .2
split_test = 1 - (split_train + split_val)

# Calculate the number of samples for training, val and test data
n_train_samples = int(len(ds) * split_train)
n_val_samples = int(len(ds) * split_val)
n_test_samples = len(ds) -  (n_train_samples + n_val_samples)

# Set Batch size
b_size = 32

import math 
# math.ceil will round up
# How many batches?
n_train_batch = math.ceil(n_train_samples/b_size)
n_val_batch = math.ceil(n_val_samples/b_size)
n_test_batch = math.ceil(n_test_samples/b_size)

print(f" - Train:\t{n_train_samples} samples \t({n_train_batch} batches)")
print(f' - Val: \t{n_val_samples} samples \t({n_val_batch} batches)')
print(f' - Test: \t{n_test_samples} samples \t({n_test_batch} batches)')

 - Train:	11833 samples 	(370 batches)
 - Val: 	3381 samples 	(106 batches)
 - Test: 	1691 samples 	(53 batches)


In [12]:
# Use take and skip to define each set
train_ds = ds.take(n_train_samples).batch(batch_size=b_size)

# Skip over the training batches and take the val batches
val_ds = ds.skip(n_train_samples). take(n_val_samples).batch(batch_size=b_size)

# Slip over the train and val batches and the remaining are the test batches
test_ds = ds.skip(n_train_samples + n_val_samples).batch(batch_size=b_size)


# Confirm the number of batches in each
print(f'There are {len(train_ds)} training batches.')
print(f'There are {len(val_ds)} validation batches.')
print(f'There are {len(test_ds)} testing batches.')                                                     

There are 370 training batches.
There are 106 validation batches.
There are 53 testing batches.


# Text Vectorization
- Going to standardize to lower case and strip punctiuation.
- Uning 'int' for output_mode assing an interger to every specific word

In [13]:
vectorizer = tf.keras.layers.TextVectorization(standardize="lower_and_strip_punctuation",
                                              output_mode='int',
                                              output_sequence_length=100)

In [14]:
ds_text = train_ds.map(lambda x,y: x)
ds_text.take(1).get_single_element()

2024-01-08 14:23:40.948945: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'And this was in truth the Death of which these bystanders spoke reverently, in low whispers you, sweet Una, gaspingly, with loud cries.',
       b'Had my eyes deceived me?',
       b"For some distance the line kept on along the gorge's brink, but at length I reached the long covered bridge where it crossed the chasm at a dizzy height.",
       b'The two forces mingling, unarmed and hand in hand, talking only how each might assist the other, the adversaries conjoined; each repenting, the one side their former cruelties, the other their late violence, they obeyed the orders of the General to proceed towards London.',
       b'I have already put off the carelessness of childhood, the unlined brow, and springy gait of early years, that they may adorn thee.',
       b'Idleness had ever been irksome to me, and now that I wished to fly from reflection, and hated my former studies, I felt great relief in being the fellow pupil with my frie

In [15]:
vectorizer.adapt(ds_text)
v_size = vectorizer.vocabulary_size()
v_size

20976

In [16]:
vocab = vectorizer.get_vocabulary()
int_to_str = {idx: word for idx, word in enumerate(vocab)}
int_to_str

{0: '',
 1: '[UNK]',
 2: 'the',
 3: 'of',
 4: 'and',
 5: 'to',
 6: 'i',
 7: 'a',
 8: 'in',
 9: 'was',
 10: 'that',
 11: 'my',
 12: 'it',
 13: 'he',
 14: 'had',
 15: 'with',
 16: 'his',
 17: 'as',
 18: 'for',
 19: 'but',
 20: 'which',
 21: 'not',
 22: 'me',
 23: 'at',
 24: 'from',
 25: 'by',
 26: 'on',
 27: 'this',
 28: 'is',
 29: 'her',
 30: 'be',
 31: 'were',
 32: 'have',
 33: 'you',
 34: 'all',
 35: 'an',
 36: 'we',
 37: 'or',
 38: 'no',
 39: 'when',
 40: 'him',
 41: 'one',
 42: 'so',
 43: 'they',
 44: 'been',
 45: 'could',
 46: 'would',
 47: 'she',
 48: 'there',
 49: 'upon',
 50: 'more',
 51: 'its',
 52: 'their',
 53: 'now',
 54: 'what',
 55: 'some',
 56: 'our',
 57: 'into',
 58: 'if',
 59: 'them',
 60: 'who',
 61: 'are',
 62: 'will',
 63: 'than',
 64: 'then',
 65: 'only',
 66: 'very',
 67: 'up',
 68: 'before',
 69: 'man',
 70: 'about',
 71: 'even',
 72: 'these',
 73: 'out',
 74: 'yet',
 75: 'your',
 76: 'time',
 77: 'did',
 78: 'any',
 79: 'old',
 80: 'said',
 81: 'might',
 82: 'li

# Adding different sub folder
import sys, os
sys.path.append( os.path.abspath("../"))

In [19]:
%load_ext autoreload
%autoreload 2

import custom_functions as cf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
